In [1]:
import os 

import pandas as pd

DATA_DIR = "../data"
RESULTS_DIR = os.path.join(DATA_DIR, "results")

In [2]:
results_df = pd.concat([pd.read_csv(os.path.join(RESULTS_DIR, f)) for f in os.listdir(RESULTS_DIR) if f[-4:] == ".csv"], ignore_index=True)

In [3]:
grouped_results_df = pd.concat([
    results_df.groupby(["imputer", "Model", "attr_group", "target"])[["AUC", "Accuracy", "Sensitivity (Recall)", "Specificity"]].mean(),
    results_df.groupby(["imputer", "Model", "attr_group", "target"])[["n_positive", "n_total"]].sum()
], axis=1).reset_index().drop("imputer", axis=1)

In [4]:
grouped_results_df["target"] = grouped_results_df["target"].str.lstrip("Dia_")
grouped_results_df = grouped_results_df.query("target == 'HFD'")
grouped_results_df

,Model,attr_group,target,AUC,Accuracy,Sensitivity (Recall),Specificity,n_positive,n_total
0,RandomForest,MICE,HFD,0.734884,0.653115,0.719603,0.560891,6829.0,11266.0
3,RandomForest,expert,HFD,0.846500,0.767353,0.801424,0.714835,6829.0,11266.0
6,RandomForest,expert_NT,HFD,0.879518,0.798953,0.847725,0.724085,6829.0,11266.0
8,RandomForest,expert_blood,HFD,0.904199,0.842696,0.757681,0.918154,2680.0,5696.0
11,RandomForest,secondary,HFD,0.947626,0.876656,0.882984,0.868739,5446.0,9591.0
18,XGBoost,expert_NT,HFD,0.873068,0.766818,0.887159,0.584099,6829.0,11266.0
22,XGBoost,secondary,HFD,0.947224,0.871235,0.876653,0.864395,5446.0,9591.0
25,LightGBM,MICE,HFD,0.756841,0.689775,0.718317,0.645370,6829.0,11266.0
26,LightGBM,expert,HFD,0.941313,0.863481,0.869819,0.853517,6829.0,11266.0
27,LightGBM,expert_blood,HFD,0.972595,0.915323,0.906531,0.929015,6829.0,11266.0


In [5]:
best_models_df = grouped_results_df.groupby(["attr_group", "target"]).apply(lambda x: x.loc[x["AUC"].idxmax()])

C:\Users\flis1\AppData\Local\Temp\ipykernel_1200\1337778183.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_models_df = grouped_results_df.groupby(["attr_group", "target"]).apply(lambda x: x.loc[x["AUC"].idxmax()])


In [6]:
best_models_df

,,Model,attr_group,target,AUC,Accuracy,Sensitivity (Recall),Specificity,n_positive,n_total
attr_group,target,,,,,,,,,
MICE,HFD,LightGBM,MICE,HFD,0.756841,0.689775,0.718317,0.645370,6829.0,11266.0
expert,HFD,LightGBM,expert,HFD,0.941313,0.863481,0.869819,0.853517,6829.0,11266.0
expert_NT,HFD,RandomForest,expert_NT,HFD,0.879518,0.798953,0.847725,0.724085,6829.0,11266.0
expert_blood,HFD,XGBoost,expert_blood,HFD,0.984853,0.929697,0.956152,0.886587,6829.0,11266.0
secondary,HFD,RandomForest,secondary,HFD,0.947626,0.876656,0.882984,0.868739,5446.0,9591.0


In [7]:
rename_cols = {"Sensitivity (Recall)": "Sensitivity", "AUC": "c-index", "target": "Predicting", "attr_group": "Attributes"}
final_selection = ["Attributes", "Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

In [8]:
compare_attr_df = best_models_df.loc[pd.IndexSlice[:, ["HFD"]], :].iloc[:-1].rename(columns=rename_cols)
best_models_df["Majority Accuracy"] = best_models_df["n_positive"]/ best_models_df["n_total"]

In [9]:
compare_attr_df = best_models_df.loc[pd.IndexSlice[:, ["HFD"]], :].rename(columns=rename_cols)
compare_attr_df["Attributes"] = ["MICE", "Expert Selection", "Expert + Blood", "Secondary"]
compare_attr_df["Majority Accuracy"] = compare_attr_df["n_positive"]/ compare_attr_df["n_total"]
compare_attr_df.loc[:, compare_attr_df.dtypes.eq("float")] = compare_attr_df.loc[:, compare_attr_df.dtypes.eq("float")].round(3)
#compare_attr_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_attrs.csv"), index=None)
compare_attr_df.loc[:, final_selection]

ValueError: Length of values (4) does not match length of index (5)

In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[pd.IndexSlice["expert", :], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_expert.csv"), index=None)
compare_expert_df.loc[:, final_selection]

In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[pd.IndexSlice["expert_blood", :], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_expert_blood.csv"), index=None)
compare_expert_df.loc[:, final_selection]

In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[[pd.IndexSlice["secondary", "HFD"]], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_secondary.csv"), index=None)
compare_expert_df.loc[:, final_selection]